In [1]:
%load_ext autoreload
%autoreload 2
from keras import optimizers
import keras.backend as KB
import keras.losses as Loss
import keras.models as M
import keras.layers as L
import tensorflow as tf
import keras.losses as losses
from keras.layers.recurrent import LSTM
import os
import sys

C:\ProgramData\Anaconda2\envs\py3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
ROOTDIR=os.path.abspath("../../")
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
data_dir=os.path.join(ROOTDIR,"Data")
sys.path.append(ROOTDIR)

In [5]:
from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_gru_generator import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back

In [6]:
import pandas as pd
files=os.listdir(data_dir)
array=pd.read_hdf(os.path.join(data_dir,files[0]))
P=preprocessor(label_encoder_dir)
array=P.process_batch(array)
action_col=array.columns=='action'
g_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':1,
        'format_':'h5',
        'ex_batch_size':10,
        'file_examples':5000,
        'max_queue_size':5,
        'Preprocessor':P,
        'static_weightage':False,
        'action_col':action_col,
        'states':10
         }

G=Generator(config=g_config)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\MuhammadUsman\\PycharmProjects\\internship.ai\\Data'

In [6]:
nn_config={'layer_shapes':[200],
           'Activation':'relu',
           'Output':6,
           'Input_shape':472,            
           'model_type':'fc',
           'HIDDEN_SIZE':128,
           'states':10,
           'loss':losses.categorical_crossentropy,
           'generator':G,
           'epochs':2
          }
my_model=Model(nn_config) 
f1=f1_score_metric(6)
my_model.model.compile(loss=weighted_cross_entropy_loss, optimizer=optimizers.Adam(lr=0.0001),metrics=["acc",f1])

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [7]:
class Model_GRU:
    def __init__(self,config):
        self.config=config
        self.hidden_size = self.config['HIDDEN_SIZE']
        self.states=self.config['states']
        self.model=self.make_model()
        self.compile_model()
    def make_model(self):
        model = M.Sequential()
        model.add(LSTM(self.hidden_size, return_sequences=True,input_shape=(self.states, self.config['Input_shape']),unroll=True))
        for i in self.config['layer_shapes']:
            model.add(L.Dense(i,activation=self.config['Activation']))
        model.add(L.Dense(self.config['Output']))
        return model
    def compile_model(self):
        self.model.compile(loss=self.config['loss'], optimizer="rmsprop")
    def fit(self):
        self.model.fit_generator(generator= self.config['generator'], 
                    steps_per_epoch  = len(G), 
                    epochs           = self.config['epochs'], 
                    verbose          = 1,
#                     validation_data  = valid_batch,
#                     validation_steps = len(valid_batch),
                    max_queue_size   = 3)

In [8]:
gru_model=Model_GRU(nn_config)

In [9]:
gru_model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 10, 128)           76928     
_________________________________________________________________
dense_3 (Dense)              (None, 10, 200)           25800     
_________________________________________________________________
dense_4 (Dense)              (None, 10, 6)             1206      
Total params: 103,934
Trainable params: 103,934
Non-trainable params: 0
_________________________________________________________________


In [10]:
from multiprocessing import Process
p1 = Process(target=G.generate)
p1.start()

In [11]:
print (G.Data.qsize())

2


In [28]:
G[0][0].shape
G[0][1].shape

(10, 10, 6)

In [12]:
gru_model.fit()

Epoch 1/2
1749/2000 [=========================>....] - ETA: 14s - loss: 7.1490

KeyboardInterrupt: 